# Import Libraries

In [2]:
import pandas as pd
import numpy as np
from keras.utils import get_file
import os

# Load the Dataset

In [5]:
# Download the data and unzip it
zip_path = get_file(
    origin="https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip",
    fname="jena_climate_2009_2016.csv.zip",
    extract=True,
)

# extract the csv path
csv_path, _ = os.path.splitext(zip_path)

In [8]:
# Load the data
df = pd.read_csv(csv_path)